# Download responses and save them to SQLite

In [1]:
!pip install --upgrade pip pandas faker sqlalchemy

In [2]:
import io
import os

import pandas
from faker import Faker
from sqlalchemy import create_engine

import citric

In [3]:
LS_URL = os.environ["LIMESURVEY_URL"]
LS_USERNAME = os.environ["LIMESURVEY_USERNAME"]
LS_PASSWORD = os.environ["LIMESURVEY_PASSWORD"]

## Create citric client

In [4]:
client = citric.Client(LS_URL, LS_USERNAME, LS_PASSWORD)

## Load survey file

In [5]:
with open("../../examples/survey.lss", "rb") as f:
    survey_id = client.import_survey(f)

## Activate survey and load fake responses

In [6]:
faker = Faker()

client.activate_survey(survey_id)
client.activate_tokens(survey_id)

data = [
    {
        "G01Q01": faker.text(max_nb_chars=100),
        "G01Q02": faker.random_int(1, 5),
        "token": faker.sha1()[:5],
        "ipaddr": faker.ipv4(),
    }
    for _ in range(100)
]

result = client.add_responses(survey_id, data)

In [7]:
with io.BytesIO() as file:
    file.write(client.export_responses(survey_id, file_format="csv"))
    file.seek(0)
    df = pandas.read_csv(
        file,
        delimiter=";",
        parse_dates=["datestamp", "startdate", "submitdate"],
        index_col="id",
    )

engine = create_engine("sqlite:///responses.db")
df.to_sql("responses", engine, if_exists="replace")

100

In [8]:
client.delete_survey(survey_id)

{'status': 'OK'}